# 1.2. Elements of Linear Algebra
_________________________________
### Key Concepts:
* Orthogonality
* Gram-Schmidt Process
* Eigenvalues and Eigenvectors

In [1]:
import numpy as np
from IPython.core.display import Image

#### Helper Functions and Global Variables

In [2]:
def generate_random_matrix(size: int = 3):
    """
    Generates Random Square Matrix of given Size with Integers (0-99)
    """
    return np.random.randint(0, 100, [size, size])

In [3]:
matrix_size = 3

# Generate a random 3x3 Matrix A
A = generate_random_matrix(matrix_size)
print("Random 3x3 Matrix A:\n", A)

# Generate another random matrix B
B = generate_random_matrix(matrix_size)
print("Random 3x3 Matrix B:\n", B)

Random 3x3 Matrix A:
 [[36 12 30]
 [81 38 95]
 [13 94 41]]
Random 3x3 Matrix B:
 [[86 85 69]
 [99 87 22]
 [46 84 99]]


## Projection

${\displaystyle \operatorname {proj} _{\mathbf {u} }(\mathbf {v} )={\frac {\langle \mathbf {u} ,\mathbf {v} \rangle }{\langle \mathbf {u} ,\mathbf {u} \rangle }}{\mathbf {u} }}$


In [4]:
def project(u, v):
    """
    Project vector u onto v
    :param u: Vector u
    :param v: Vector v
    :return:
    """
    return (np.dot(v, u) / np.dot(u, u)) * u

In [5]:
# Project the first row of A onto first row of B
proj_A_B = project(A[0], B[0])
print("A1 Projected onto B1")
print(proj_A_B)

A1 Projected onto B1
[95.16923077 31.72307692 79.30769231]


## Gram-Schmidt Process


In [6]:
Image(url= "https://wikimedia.org/api/rest_v1/media/math/render/svg/c14bc18275503c8b2593498757740976cf0e078a", height=200)

In [7]:
def gram_schmidt(u, v, k):
    """
    Gram-Schmidt Process for step k
    :param u: First set of vectors
    :param v: Second set of vectors
    :param k: Step
    :return e: Orthonormal vector
    """
    # Index starts at 0 (not 1) in python
    k = k-1

    # Get v_k
    v_k = v[:, k]

    # Calculate projection matrices
    proj_sum = np.zeros(v_k.shape)
    # Iterate from first to second-to-last element
    for j in range(0, k):
        # Project u_j onto v_k
        proj = project(u[:, j], v_k)

        # Update the projections sum
        proj_sum += proj

    u_k = (v_k - proj_sum)

    # Normalize
    e_k = (u_k / np.linalg.norm(u_k))

    return e_k

In [8]:
def gram_schmidt_matrix(A):
    """
    Find the Gram-Schmidt of a Matrix
    """
    A_gs = np.zeros(A.shape)
    # Iterate through each column
    for k in range(matrix_size):
        proj_sum = np.zeros(A[:, k].shape)
        # Iterate through each column to the left of k
        for j in range(k):
             proj_sum += project(A[:, j], A[:, k])

        # Get new column vector
        gs_col = (A[:, k] - proj_sum)

        # Normalize
        A_gs[:, k] = (gs_col / np.linalg.norm(gs_col))

    return A_gs

In [9]:
# Calculate the Gram-Schmidt of A
gs_A = gram_schmidt_matrix(A)

print("Gram-Schmidt Matrix of A:")
print(gs_A)

Gram-Schmidt Matrix of A:
[[ 0.40183978 -0.1055817  -0.36827617]
 [ 0.9041395  -0.1116622  -0.48719873]
 [ 0.14510881  0.98812148 -0.79183967]]
